# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/erinberardi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/erinberardi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erinberardi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# your code here
sent = pd.read_csv('/Users/erinberardi/Downloads/Sentiment140.csv',encoding='latin-1')
sample=sent.sample(n=10000, axis=0)
sample.shape
sample.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1098225,4,1970462512,Sat May 30 03:58:07 PDT 2009,NO_QUERY,brandonsalomon,Time for golf.... and time to win http://mylo...
1541408,4,2180797158,Mon Jun 15 10:57:31 PDT 2009,NO_QUERY,_Annet,@ashthebeliever nice song! It's my fave song a...
916131,4,1753253655,Sun May 10 00:46:32 PDT 2009,NO_QUERY,kitz007,@vimoh that was funny n almost right
602237,0,2221341934,Thu Jun 18 05:11:53 PDT 2009,NO_QUERY,sofiejenkinson,"@evacaiden whoops, i'm a dick. oh man, i'm goi..."
1398111,4,2054058391,Sat Jun 06 06:26:59 PDT 2009,NO_QUERY,waynehager,@anthonycoxjr that's easy! Starbucks ANDDunki...


In [3]:
sample.columns =['target','id','date','flag','user','text']
sample.head()

,target,id,date,flag,user,text
1098225,4,1970462512,Sat May 30 03:58:07 PDT 2009,NO_QUERY,brandonsalomon,Time for golf.... and time to win http://mylo...
1541408,4,2180797158,Mon Jun 15 10:57:31 PDT 2009,NO_QUERY,_Annet,@ashthebeliever nice song! It's my fave song a...
916131,4,1753253655,Sun May 10 00:46:32 PDT 2009,NO_QUERY,kitz007,@vimoh that was funny n almost right
602237,0,2221341934,Thu Jun 18 05:11:53 PDT 2009,NO_QUERY,sofiejenkinson,"@evacaiden whoops, i'm a dick. oh man, i'm goi..."
1398111,4,2054058391,Sat Jun 06 06:26:59 PDT 2009,NO_QUERY,waynehager,@anthonycoxjr that's easy! Starbucks ANDDunki...


In [4]:
def clean_up(s):
    s = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', ' ', s)
    s = s.translate ({ord(c): " " for c in """!@#$%^&*()[]{}'";:,./<>?\|`-~=_+"""})
    s = re.sub('\d+',' ',s)
    s = s.lower()

    return s

In [5]:
def tokenize(s):
    tokens = word_tokenize(s)
    return tokens

In [6]:
def stem_and_lemmatize(l):
    l_stem_lem = []
    ps = PorterStemmer() 
    lm = WordNetLemmatizer()
    for w in l: 
        w=ps.stem(w)
        w=lm.lemmatize(w)
        l_stem_lem.append(w)
    return l_stem_lem

In [7]:
def remove_stopwords(l):
    stop_words = set(stopwords.words('english')) 
    return [word for word in l if word not in stop_words]
  

In [ ]:
sample.head()


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [ ]:
sample5=sent.sample(n=5, axis=0)
sample5

In [8]:
# your code here
# apply(clean_up)
# apply(tokenize)
# apply(stem_and_lemmatize)
# apply(remove_stopwords)
#creating new column with cleaned,tokenized, stemmed and lemmed, no stopwords

sample["text_processed"]=sample["text"].apply(clean_up)
print('1 done')
sample["text_processed"]=sample["text_processed"].apply(tokenize)
print('2 done')
sample["text_processed"]=sample["text_processed"].apply(stem_and_lemmatize)
print('3 done')
sample["text_processed"]=sample["text_processed"].apply(remove_stopwords)

sample.head()

1 done
2 done
3 done


,target,id,date,flag,user,text,text_processed
1098225,4,1970462512,Sat May 30 03:58:07 PDT 2009,NO_QUERY,brandonsalomon,Time for golf.... and time to win http://mylo...,"[time, golf, time, win]"
1541408,4,2180797158,Mon Jun 15 10:57:31 PDT 2009,NO_QUERY,_Annet,@ashthebeliever nice song! It's my fave song a...,"[ashthebeliev, nice, song, fave, song, kill, n..."
916131,4,1753253655,Sun May 10 00:46:32 PDT 2009,NO_QUERY,kitz007,@vimoh that was funny n almost right,"[vimoh, wa, funni, n, almost, right]"
602237,0,2221341934,Thu Jun 18 05:11:53 PDT 2009,NO_QUERY,sofiejenkinson,"@evacaiden whoops, i'm a dick. oh man, i'm goi...","[evacaiden, whoop, dick, oh, man, go, shrewsbu..."
1398111,4,2054058391,Sat Jun 06 06:26:59 PDT 2009,NO_QUERY,waynehager,@anthonycoxjr that's easy! Starbucks ANDDunki...,"[anthonycoxjr, easi, starbuck, anddunkin, donut]"


## Looking at Stemming and Lemmatizing

In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
what = stem_and_lemmatize(['good','better','best'])
what

In [ ]:
ps = PorterStemmer() 
lst = ['caring','better','stemming']
stem = []
for i in lst:
    s = ps.stem(i)
    stem.append(s)
stem

In [ ]:
lm = WordNetLemmatizer()
lemm = []
for i in lst:
    s = lm.lemmatize(i)
    lemm.append(s)
lemm

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
from nltk.probability import FreqDist


In [12]:
# your code here

# List of list of all words
list_of_words = sample["text_processed"].tolist()
list_of_words

# Create one list of all words
bow = []
for lists in list_of_words:
    for word in lists:
        bow.append(word)
#print(bow)

# Create frequency with words
freq_dist = FreqDist(bow)
top_5000 = freq_dist.most_common(5000)

In [ ]:
#top_5000

In [13]:
# separating list of words and frequency
word_features, freq = [[x for x,y in top_5000],
      [y for x,y in top_5000]]

# List of just words
# print(word_features)
# print(freq)



### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [15]:
# your code here
# word_features=list(bow)
# word_features

def find_features(lst, bow):
    word_features=list(bow)
    words = set(lst)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

word_features
features=[]
for i,l in enumerate(sample["text_processed"]):
    s=[find_features(l,bow),sample["target"].iloc[i]]
    z=tuple(s)
    features.append(z)
len(features)

10000

In [ ]:
#features -- not printing to keep the lab shorter


In [ ]:
#featuresets = [(find_features(word_features), category) for (rev, category) in features]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [16]:
# Split into train and test
train_set, test_set = features[5000:], features[:5000]

In [17]:
# your code here

classifier = nltk.NaiveBayesClassifier.train(train_set)

In [18]:
classifier.show_most_informative_features()

Most Informative Features
                     sad = True                0 : 4      =     17.1 : 1.0
                     ugh = True                0 : 4      =     13.0 : 1.0
                     die = True                0 : 4      =     13.0 : 1.0
               goodnight = True                4 : 0      =     13.0 : 1.0
                    sick = True                0 : 4      =     12.8 : 1.0
                       v = True                4 : 0      =     10.3 : 1.0
                   broke = True                0 : 4      =      9.7 : 1.0
                    hate = True                0 : 4      =      8.6 : 1.0
                 horribl = True                0 : 4      =      8.4 : 1.0
                    pain = True                0 : 4      =      8.4 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [19]:
# your code here
nltk.classify.accuracy(classifier, test_set)

0.7048

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here